# Introduction to GIS - Using GeoPandas

- Dennis Milechin, P.E., GISP
- <a href="http://rcs.bu.edu">Research Computing Services</a>



# Outline

1. Download tutorial data.
1. Explore GeoPandas
1. Coordinate Reference System (CRS)
1. Calculate Spatial Attributes
1. Spatial Processing
1. Saving GIS Data


# 1. Download Tutorial Data


Below we will use linux `wget` and `unzip` to get the data we need.

If oyu are not using Google Collab, you can download the data directly using this link:
http://rcs.bu.edu/examples/gis/tutorials/python_geopandas/tutorial_files.zip

In [ ]:
!wget http://rcs.bu.edu/examples/gis/tutorials/python_geopandas/tutorial_files.zip

In [ ]:
!unzip tutorial_files.zip

# 2. Explore GeoPandas



> GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types.

<sup>source: https://geopandas.org/index.html</sup>

__Note:__ GeoPandas is for processing vector data.

User Guide: https://geopandas.org/docs/user_guide.html

### Vector

<img src="https://github.com/milechin/tut_geopandas/blob/main/images/vector_data.png?raw=true" alt="compass" style="width:500px"/>  

### GeoPandas - Dependencies

- __numpy__
- __pandas__
- __shapely__  - Spatial Operations
- __pyogrio__  - Reading and Writing Files
- __pyproj__ - Projection definition and transformations

Source: https://geopandas.org/getting_started/install.html#dependencies

### Read Shapefile


In [ ]:
import geopandas

mbta_stations = geopandas.read_file("./mbta_rapid_transit/MBTA_NODE.shp")
mbta_stations.head()

- "geometry" column contains spatial information.
- rows - known as "features" or "records".
- columns - known as "attributes" or "fields".

Let's look at types for each column:

In [ ]:
mbta_stations.dtypes

### Plotting

In [ ]:
mbta_stations.plot()

In [ ]:
mbta_stations.plot(column="LINE", legend=False)

### Load Polyline Data

In [ ]:
mbta_lines = geopandas.read_file("./mbta_rapid_transit/MBTA_ARC.shp")
mbta_lines.head()

In [ ]:
mbta_lines.LINE.unique()

In [ ]:
mbta_lines.plot(column="LINE", cmap='tab10')

### Select By Attribute

In [ ]:
mbta_lines.LINE.unique()

In [ ]:
green_line = mbta_lines[mbta_lines["LINE"]=="GREEN"]
green_line

In [ ]:
green_line.geometry.plot(cmap='tab10')

### Color by Attribute

<sup>This example obtained from : https://www.earthdatascience.org/courses/scientists-guide-to-plotting-data-in-python/plot-spatial-data/customize-vector-plots/python-customize-map-legends-geopandas/</sup>


In [ ]:
import matplotlib.pyplot as plt
subway_colors = {'SILVER': 'grey',
               'ORANGE': 'orange',
               'GREEN': 'green',
               'RED': 'red',
                'BLUE':'blue'}

In [ ]:
fig, ax = plt.subplots()
for ctype, data in mbta_lines.groupby('LINE'):
    color = subway_colors[ctype]
    data.plot(color=color, ax=ax, label=ctype)

ax.legend(bbox_to_anchor=(1.5, .5), prop={'size': 12})
ax.set(title='MBTA Subway Lines')
ax.set_axis_off()

mbta_stations.plot(ax=ax, color="black", zorder=2)
plt.show()

### Load data from Geodatabase

- *Geodatabase* is an ESRI developed model.  Similiar open source is *GeoPackage*.
- Both act like a database and will contain multiple layers.

In [ ]:
gdp_path = "./tlgdb_2019_a_25_ma.gdb"
geopandas.list_layers(gdp_path)

In [ ]:
ma_block = geopandas.read_file(gdp_path, layer="Block_Group")

In [ ]:
ma_block.head()

In [ ]:
ma_block.plot(column="NAMELSAD")

###  Determine vector data type
    

In [ ]:
print("mbta_stations")
print(mbta_stations.geometry.type.unique())

print("\nmbta_lines")
print(mbta_lines.geometry.type.unique())

print("\nma_block")
print(ma_block.geometry.type.unique())

### Plotting Multiple Layers on a Map
Let's plot all three layers on a single map.

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

for ctype, data in mbta_lines.groupby('LINE'):
    color = subway_colors[ctype]
    data.plot(color=color, ax=ax, label=ctype)

ax.legend(bbox_to_anchor=(1.5, .5), prop={'size': 12})
ax.set(title='MBTA Subway Lines')
ax.set_axis_off()

mbta_stations.plot(ax=ax, color="black", zorder=1)  # MBTA Stations layer
ma_block.plot(ax=ax) # MA Group Block layer

plt.show()

In [ ]:
print("mbta_lines CRS")
print(mbta_lines.crs)
print("\nmbta_stations CRS")
print(mbta_stations.crs)
print("\nma_block CRS")
print(ma_block.crs)

# 3. Coordinate Reference System (CRS)

__Geographic coordinate system__ and __projections__ are known as Coordinate Reference System (__CRS__) in GIS.

### GeoPandas CRS Functions

<table>
<colgroup>
<col style="width: 10%" />
<col style="width: 90%" />
</colgroup>
<tbody>
<tr ><td><p><a href="https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.crs.html#geopandas.GeoDataFrame.crs">GeoDataFrame.crs</a></p></td>
<td><p>The Coordinate Reference System (CRS) represented as a pyproj.CRSobject.</p></td>
</tr>
<tr><td><p><a href="https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.to_crs.html#geopandas.GeoDataFrame.to_crs">GeoDataFrame.to_crs()</a></p></td>
<td><p>Transform geometries to a new coordinate reference system.</p></td>
</tr>
<tr><td><p><a href="https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.set_crs.html#geopandas.GeoDataFrame.set_crs">GeoDataFrame.set_crs()</a></p></td>
    <td><p>Set the Coordinate Reference System (CRS) of the GeoDataFrame. <b>### Only use if data does not have CRS defined! ###</b></p></td>
</tr>
</tbody>
</table>



### GeoPandas Accepted Definitions


- CRS WKT string
- __An authority string (i.e. “epsg:4326”)__
- __An EPSG integer code (i.e. 4326)__
- A pyproj.CRS
- An object with a to_wkt method.
- PROJ string
- Dictionary of PROJ parameters
- PROJ keyword arguments for parameters
- JSON string with PROJ parameters

<sup> source: https://geopandas.org/docs/user_guide/projections.html </sup>

### EPSG Codes

You can find library of EPSG codes at: https://spatialreference.org/

### Let's Play Around with CRS

In [ ]:
world = geopandas.read_file("ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
world.crs

In [ ]:
world.head()

In [ ]:
world.plot()

### Projections

- Lets convert to "WGS 84 / Australian Antarctic Polar Stereographic" projection.

In [ ]:
world_3031 = world.to_crs("epsg:3031")
world_3031.crs

In [ ]:
world_3031.plot()

In [ ]:
antartica_3031 = world_3031[world_3031["ADMIN"] == "Antarctica"]
antartica_3031.plot()

In [ ]:
antartica = world[world["ADMIN"] == "Antarctica"]
antartica.plot()

## Back to our Example

In [ ]:
print("mbta_lines CRS")
print(mbta_lines.crs)
print("\nmbta_stations CRS")
print(mbta_stations.crs)
print("\nma_block CRS")
print(ma_block.crs)

In [ ]:
ma_block_proj = ma_block.to_crs(mbta_lines.crs)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for ctype, data in mbta_lines.groupby('LINE'):
    color = subway_colors[ctype]
    data.plot(color=color,
              ax=ax,
              label=ctype)
ax.legend(bbox_to_anchor=(0.5, .5), prop={'size': 12})
ax.set(title='MBTA Subway Lines')
ax.set_axis_off()
mbta_stations.plot(ax=ax, color="black", zorder=1)
ma_block_proj.plot(ax=ax,column="NAMELSAD", alpha=0.5)
plt.show()

In [ ]:
bounds = mbta_lines.to_crs("EPSG:4269").total_bounds

In [ ]:
ma_block = geopandas.read_file(gdp_path, layer="Block_Group", bbox=tuple(bounds))
ma_block.plot(column="NAMELSAD", alpha=0.5)

In [ ]:
# Need to convert to Mass Projection.
ma_block_proj = ma_block.to_crs(mbta_lines.crs)

In [ ]:
fig, ax = plt.subplots()
for ctype, data in mbta_lines.groupby('LINE'):
    color = subway_colors[ctype]
    data.plot(color=color, ax=ax, label=ctype)
ax.legend(bbox_to_anchor=(1, .5), prop={'size': 12})
ax.set(title='MBTA Subway Lines')
ax.set_axis_off()
mbta_stations.plot(ax=ax, color="black", zorder=1, alpha=1)
ma_block_proj.plot(ax=ax, zorder=0, column="NAMELSAD", alpha=0.25, edgecolor="black")
plt.show()

# 4. Calculate Spatial Attributes


- This is focused on just one layer.
- What is the length/area of each feature?
- What is the distance from one feature to another?

### Area

In [ ]:
ma_block.columns

In [ ]:
ma_block.area

In [ ]:
ma_block.crs

In [ ]:
ma_block_proj = ma_block.to_crs("EPSG:26986")
ma_block_proj.crs

In [ ]:
ma_block_proj.area

In [ ]:
ma_block_proj["area"] = ma_block_proj.area
ma_block_proj.head()

What are the units of *area*?

### Perimeter

In [ ]:
ma_block_proj["perimeter_m"] = ma_block_proj.length
ma_block_proj.head()

### Centroid

In [ ]:
ma_block_proj["centroid"] = ma_block_proj.centroid
ma_block_proj.head()

In [ ]:
ma_block_proj.dtypes

Note that there are two geometry columns!

In [ ]:
ma_block_proj.plot(column="NAMELSAD")

In [ ]:
ma_block_proj.geometry.name

In [ ]:
ma_block_proj = ma_block_proj.set_geometry('centroid')
ma_block_proj.plot(column="NAMELSAD")

In [ ]:
ma_block_proj = ma_block_proj.set_geometry('geometry')

# 5. Spatial Processing


Spatial Processing allows us to ask questions regarding two layers.
- What are the nearest block groups to Kenmore Station?
- Which block groups have an MBTA station?
- Which block groups have MBTA lines that pass through them?
- Which block groups are within one mile radius of MBTA tracks?

### Distance
What are the nearest block groups to Kenmore station?

In [ ]:
kenmore_station = mbta_stations.loc[mbta_stations['STATION'] == "Kenmore"]

type(kenmore_station.geometry.iloc[0])

In [ ]:
ma_block_proj["dist_to_kenmore_m"] = ma_block_proj.distance(kenmore_station.geometry.iloc[0])
ma_block_proj.head(n=3)

In [ ]:
near_kenmore = ma_block_proj.sort_values(['dist_to_kenmore_m'], ascending=[True]).head(n=5)

base = near_kenmore.plot(column="NAMELSAD")
kenmore_station.plot(ax=base, zorder=2, color="black")

### Within
Which Census Block Groups have an MBTA Station?

In [ ]:
ma_block_proj["w_station"] = ma_block_proj.geometry.apply(lambda x: mbta_stations.geometry.within(x).any())
ma_block_proj.head(n=3)

In [ ]:
base = ma_block_proj.plot(column="w_station", legend=True, categorical=True, cmap="Paired")
mbta_lines.plot(ax=base, color="black", alpha=0.5)

### Spatial Join
- Joining the attributes of two layers based on their spatial location to each other.
- Spatial joins are normally done with point and polygon layers, line and polygon layers, or line and line layers.

__Contains__ - Do any points exist inside the polygon?
<img src="https://github.com/milechin/tut_geopandas/blob/main/images/sjoin_contains.png?raw=true" alt="compass" style="width:500px"/>    


Note: Since three points exist within the polygon, the ouput of this process will result in 3 identical polygons but will have different attributes representing each point.

### Example

In [ ]:
print("ma_block_proj")
print(ma_block_proj.columns)
print("\nmbta_stations")
print(mbta_stations.columns)

In [ ]:
blocks_w_stations = geopandas.sjoin(ma_block_proj, mbta_stations, how="inner", predicate='contains')
blocks_w_stations.head(n=2)

In [ ]:
blocks_w_stations["LINE"].unique()

In [ ]:
subway_colors = {'SILVER': 'grey',
               'ORANGE': 'orange',
               'GREEN': 'green',
               'RED': 'red',
                'BLUE':'blue',
                'GREEN/ORANGE': 'yellow',
                'BLUE/GREEN': 'yellow',
                'ORANGE/RED': 'yellow',
                'GREEN/RED': 'yellow',
                'BLUE/ORANGE': 'yellow'}

In [ ]:
fig, ax = plt.subplots()
for ctype, data in blocks_w_stations.groupby('LINE'):
    color = subway_colors[ctype]
    data.plot(color=color, ax=ax, label=ctype)
plt.show()

In [ ]:
print("ma_block_proj")
print(ma_block_proj.columns)
print(ma_block_proj.shape)
print("\nmbta_stations")
print(mbta_stations.columns)
print(mbta_stations.shape)

In [ ]:
# Note to Dennis, do "right" first
blocks_w_stations = geopandas.sjoin(ma_block_proj, mbta_stations, how="left", predicate='contains')
print(blocks_w_stations.shape)
blocks_w_stations.columns

```python
geopandas.sjoin(ma_block_proj, mbta_stations, how= , predicate="contains")
```

__how__ options:
- __left__ - Retain all __ma_block_proj__ features and append __mbta_stations__ attributes where the result of "predicate" is True.
- __inner__ - Retain only __ma_block_proj__ features where the result of "predicate" is True and append __mbta_stations__ attributes to __ma_block_proj__ that satisfy the "predicate".
- __right__ - Retain all __mbta_stations__ features and append __ma_block_proj__ where the result of "predicate" is True.

<sup> Source: https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#binary-predicate-joins</sup>

In [ ]:
blocks_w_stations[["GEOID", "STATION"]].groupby(by="GEOID").count().sort_values(by="STATION", ascending=False ).head(15)

In [ ]:
blocks_w_stations[blocks_w_stations["GEOID"] == "250259813002"]["STATION"]

###  Intersect
Do any line segments intersect the polygon?
<img src="https://github.com/milechin/tut_geopandas/blob/main/images/sjoin_intersect.png?raw=true" alt="compass" style="width:500px"/>    

What Census Group Blocks have MBTA Rapid transit tracks going through them?

In [ ]:
blocks_w_lines = geopandas.sjoin(ma_block_proj, mbta_lines, how="inner", predicate='intersects')

base = blocks_w_lines.plot(column="NAMELSAD", alpha=0.5)
mbta_lines.plot(ax=base, color="black")

GeoPandas uses the following operators:
- __contains__ -  Returns True if no points of other lie in the exterior of the object and at least one point of the interior of other lies in the interior of object.
- __crosses__ - Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.
- __disjoint__ - Returns True if the boundary and interior of the object do not intersect at all with those of the other.
- __intersects__ - Returns True if the boundary or interior of the object intersect in any way with those of the other. In other words, geometric objects intersect if they have any boundary or interior point in common.
- __overlaps__ - Returns True if the geometries have more than one but not all points in common, have the same dimension, and the intersection of the interiors of the geometries has the same dimension as the geometries themselves.
- __touches__ - Returns True if the objects have at least one point in common and their interiors do not intersect with any part of the other.
- __within__ - Returns True if the object’s boundary and interior intersect only with the interior of the other (not its boundary or exterior).

### Buffer, Dissolve, Overlay
What Census Group Blocks are within 800 meters of MBTA Rapid Transit tracks?

In [ ]:
mbta_lines["buffer"] = mbta_lines.buffer(800)
mbta_lines.head()

In [ ]:
mbta_lines = mbta_lines.set_geometry("buffer")
print(mbta_lines.shape)
mbta_lines.plot(column="LINE", edgecolor="black")

In [ ]:
mbta_buffers = mbta_lines.dissolve(by="LINE")
mbta_buffers.head()

In [ ]:
mbta_buffers["LINE"] = mbta_buffers.index
print(mbta_buffers.shape)
mbta_buffers.plot(edgecolor="black", column="LINE")

In [ ]:
ma_block_proj.columns

In [ ]:
blocks_within_buffer = geopandas.overlay(ma_block_proj, mbta_buffers, how="intersection")
blocks_within_buffer.head(n=3)

In [ ]:
blocks_within_buffer.plot(column="LINE", edgecolor="black", linewidth=0.2)

In [ ]:
blocks_within_buffer["area_in_buffer"] = blocks_within_buffer.area

In [ ]:
blocks_within_buffer[["area", "area_in_buffer"]].head(n=3)

In [ ]:
blocks_within_buffer["area_ratio"] = blocks_within_buffer["area_in_buffer"]/blocks_within_buffer["area"]
blocks_within_buffer[["area", "area_in_buffer", "area_ratio"]].head(n=3)

More information about overlay function: https://geopandas.org/docs/user_guide/set_operations.html

# 6. Saving GIS Data

We will save the data as a geopackage file, an open source file format that can store multiple layers.  For more information about Geopackage can be found here:

https://www.geopackage.org/

In [ ]:
blocks_w_stations.to_file("tut_output.gpkg", driver="GPKG", layer="blocks_w_stations")

The error is not clear, but it is complaining there are two "geometry" fields in this layer.  Most file formats only support one geometry field.

In [ ]:
blocks_w_stations.dtypes

In [ ]:
# Set the appropriate geometry column you want to save
blocks_w_stations.set_geometry("geometry", inplace=True)

# Remove the other geometry column
blocks_w_stations.drop(columns=["centroid"], inplace=True)

# Save the layer in a GeoPackage
blocks_w_stations.to_file("tut_output.gpkg", driver="GPKG", layer="blocks_w_stations")

In [ ]:
geopandas.list_layers("tut_output.gpkg")

In [ ]:
blocks_within_buffer.set_geometry("geometry", inplace=True)
blocks_within_buffer.drop(columns=["centroid"], inplace=True)

ma_block_proj.set_geometry("geometry", inplace=True)
ma_block_proj.drop(columns=["centroid"], inplace=True)

mbta_lines.set_geometry("geometry", inplace=True)
mbta_lines.drop(columns=["buffer"], inplace=True)

blocks_w_lines.set_geometry("geometry", inplace=True)
blocks_w_lines.drop(columns=["centroid"], inplace=True)

In [ ]:
blocks_within_buffer.to_file("tut_output.gpkg", driver="GPKG", layer="blocks_within_buffer")
ma_block_proj.to_file("tut_output.gpkg", driver="GPKG", layer="ma_block_proj")
mbta_lines.to_file("tut_output.gpkg", driver="GPKG", layer="mbta_lines")
blocks_w_lines.to_file("tut_output.gpkg", driver="GPKG", layer="blocks_w_lines")

In [ ]:
geopandas.list_layers("tut_output.gpkg")

# Questions?


## Data Sources
- Mass. Census Boundary Data - https://www2.census.gov/geo/tiger/TGRGDB19/tlgdb_2019_a_25_ma.gdb.zip  
- MBTA Rapid Transit - https://www.mass.gov/info-details/massgis-data-mbta-rapid-transit
- Natural Earth Political Boundaries - https://www.naturalearthdata.com/downloads/110m-cultural-vectors/